In [1]:
import json
import requests
import shutil
from typing import Dict, Any
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from pydantic import BaseModel, Field

In [23]:
# LangSmith imports for observability
from langsmith import Client
from langchain.callbacks import LangChainTracer
from langchain.callbacks.manager import CallbackManager
import os

In [ ]:
def setup_langsmith():
    # Option 1: Set via environment variables (recommended)
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = "llama-agent-observability"  # Your project name
    
    # Option 2: Set your API key here (or better yet, use environment variable)
    os.environ["LANGCHAIN_API_KEY"] = ""
    
    # If you haven't set the API key as environment variable, uncomment and set it:
    if "LANGCHAIN_API_KEY" not in os.environ:
        print("⚠️  LANGCHAIN_API_KEY not found in environment variables")
        print("   Please set it up:")
        print("   1. Go to https://smith.langchain.com/")
        print("   2. Sign up/Login")
        print("   3. Get your API key from settings")
        print("   4. Set environment variable: export LANGCHAIN_API_KEY='your-key'")
        print("   5. Or uncomment the line in the code and add your key")
        return False
    
    try:
        # Test LangSmith connection
        client = Client()
        print("✓ LangSmith connection successful!")
        print(f"✓ Project: {os.environ.get('LANGCHAIN_PROJECT', 'default')}")
        return True
    except Exception as e:
        print(f"❌ LangSmith setup failed: {e}")
        return False

In [2]:
class WikipediaSearchInput(BaseModel):
    query: str = Field(description="Search query for Wikipedia")

In [3]:
class WikipediaTool(BaseTool):
    name = "wikipedia_search"
    description = "Search Wikipedia for information about a topic. Input should be a search query string."
    args_schema = WikipediaSearchInput
    
    def _run(self, query: str) -> str:
        try:
            url = "https://en.wikipedia.org/api/rest_v1/page/summary/" + query.replace(" ", "_")
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                return f"Title: {data.get('title', 'N/A')}\nSummary: {data.get('extract', 'No summary available')}"
            else:
                return f"Wikipedia search failed with status code: {response.status_code}"
        except Exception as e:
            return f"Error searching Wikipedia: {str(e)}"


In [10]:
#!/usr/bin/env python3

import json
import requests
import shutil
from typing import Dict, Any
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from pydantic import BaseModel, Field

# Tool 1: Wikipedia API Query
class WikipediaSearchInput(BaseModel):
    query: str = Field(description="Search query for Wikipedia")

class WikipediaTool(BaseTool):
    name = "wikipedia_search"
    description = "Search Wikipedia for information about a topic. Input should be a search query string."
    args_schema = WikipediaSearchInput
    
    def _run(self, query: str) -> str:
        try:
            url = "https://en.wikipedia.org/api/rest_v1/page/summary/" + query.replace(" ", "_")
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                return f"Title: {data.get('title', 'N/A')}\nSummary: {data.get('extract', 'No summary available')}"
            else:
                return f"Wikipedia search failed with status code: {response.status_code}"
        except Exception as e:
            return f"Error searching Wikipedia: {str(e)}"

# Tool 2: arXiv API Query
class ArxivSearchInput(BaseModel):
    query: str = Field(description="Search query for arXiv papers")

class ArxivTool(BaseTool):
    name = "arxiv_search"
    description = "Search arXiv for academic papers. Input should be a search query string. Returns top 5 results."
    args_schema = ArxivSearchInput
    
    def _run(self, query: str) -> str:
        max_results = 5  # Fixed to 5 results
        try:
            import xml.etree.ElementTree as ET
            url = f"http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results=5"
            response = requests.get(url)
            
            if response.status_code == 200:
                root = ET.fromstring(response.content)
                entries = root.findall('{http://www.w3.org/2005/Atom}entry')
                
                results = []
                for entry in entries[:5]:
                    title = entry.find('{http://www.w3.org/2005/Atom}title').text.strip()
                    authors = [author.find('{http://www.w3.org/2005/Atom}name').text 
                             for author in entry.findall('{http://www.w3.org/2005/Atom}author')]
                    summary = entry.find('{http://www.w3.org/2005/Atom}summary').text.strip()
                    
                    results.append(f"Title: {title}\nAuthors: {', '.join(authors)}\nSummary: {summary[:200]}...\n")
                
                return "\n".join(results) if results else "No papers found"
            else:
                return f"arXiv search failed with status code: {response.status_code}"
        except Exception as e:
            return f"Error searching arXiv: {str(e)}"


In [17]:
# Tool 3: Celsius to Fahrenheit Converter - FIXED
class TempConvertInput(BaseModel):
    celsius: str = Field(description="Temperature in Celsius to convert to Fahrenheit")

class CelsiusToFahrenheitTool(BaseTool):
    name = "celsius_to_fahrenheit"
    description = "Convert temperature from Celsius to Fahrenheit. Input should be a number representing degrees Celsius."
    args_schema = TempConvertInput
    
    def _run(self, celsius: str) -> str:
        try:
            # Convert string to float
            celsius_float = float(celsius)
            fahrenheit = (celsius_float * 9/5) + 32
            return f"{celsius_float}°C = {fahrenheit}°F"
        except ValueError:
            return f"Error: '{celsius}' is not a valid number"
        except Exception as e:
            return f"Error converting temperature: {str(e)}"

In [18]:
# Tool 4: Save Text to File
class SaveTextInput(BaseModel):
    text_and_filename: str = Field(description="Text content and filename separated by '|'. Format: 'text content|filename.txt'")

class SaveTextTool(BaseTool):
    name = "save_text_file"
    description = "Save text content to a file. Input format: 'text content|filename.txt' (separate text and filename with |)"
    args_schema = SaveTextInput
    
    def _run(self, text_and_filename: str) -> str:
        try:
            if '|' not in text_and_filename:
                return "Error: Please provide input in format 'text content|filename.txt'"
            
            parts = text_and_filename.split('|', 1)
            text = parts[0].strip()
            filename = parts[1].strip()
            
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(text)
            return f"Successfully saved text to {filename}"
        except Exception as e:
            return f"Error saving file: {str(e)}"


In [19]:
# Tool 5: Copy File
class CopyFileInput(BaseModel):
    source_and_dest: str = Field(description="Source and destination file paths separated by '|'. Format: 'source_path|destination_path'")

class CopyFileTool(BaseTool):
    name = "copy_file"
    description = "Make a copy of a file. Input format: 'source_path|destination_path' (separate paths with |)"
    args_schema = CopyFileInput
    
    def _run(self, source_and_dest: str) -> str:
        try:
            if '|' not in source_and_dest:
                return "Error: Please provide input in format 'source_path|destination_path'"
            
            parts = source_and_dest.split('|', 1)
            source = parts[0].strip()
            destination = parts[1].strip()
            
            shutil.copy2(source, destination)
            return f"Successfully copied {source} to {destination}"
        except Exception as e:
            return f"Error copying file: {str(e)}"


In [ ]:
def main():
    # Initialize Ollama LLM
    llm = Ollama(model="llama3")
    
    # Create tools list
    tools = [
        WikipediaTool(),
        ArxivTool(),
        CelsiusToFahrenheitTool(),
        SaveTextTool(),
        CopyFileTool()
    ]
    
    # Initialize agent
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=3,  # Limit iterations
        early_stopping_method="generate"  # Stop when answer generated
    )
    
    # Test queries
    test_queries = [
        "What is quantum computing? Search Wikipedia for information.",
        "Find recent papers about machine learning on arXiv",
        "Convert 25 degrees Celsius to Fahrenheit",
        "Save the text 'Hello, this is a test file!' to a file called test.txt",
        "Copy the file test.txt to backup_test.txt"
    ]
    
    print("Testing custom tools with Llama 3...")
    print("=" * 50)
    
    for i, query in enumerate(test_queries, 1):
        print(f"\nTest {i}: {query}")
        print("-" * 30)
        try:
            result = agent.run(query)
            print(f"Result: {result}")
        except Exception as e:
            print(f"Error: {str(e)}")
        print("=" * 50)

In [21]:
if __name__ == "__main__":
    main()

Testing custom tools with Llama 3.1...

Test 1: What is quantum computing? Search Wikipedia for information.
------------------------------


> Entering new AgentExecutor chain...
Thought: It seems like a great idea to start with understanding what quantum computing is, as it's a fundamental concept in modern computer science.

Action: wikipedia_search
Action Input: "quantum computing"
Observation: Title: Quantum computing
Summary: A quantum computer is a computer that exploits quantum mechanical phenomena. On small scales, physical matter exhibits properties of both particles and waves, and quantum computing takes advantage of this behavior using specialized hardware. Classical physics cannot explain the operation of these quantum devices, and a scalable quantum computer could perform some calculations exponentially faster than any modern "classical" computer. Theoretically a large-scale quantum computer could break some widely used encryption schemes and aid physicists in performing 

KeyboardInterrupt: 

In [40]:
def create_agent_with_observability():
    
    # Initialize Ollama LLM
    llm = Ollama(model="llama3", temperature=0.1)
    
    # Create tools list
    tools = [
        WikipediaTool(),
        ArxivTool(),
        CelsiusToFahrenheitTool(),
        SaveTextTool(),
        CopyFileTool()
    ]
    
    # Setup callback manager for LangSmith tracing
    tracer = LangChainTracer(project_name="llama-agent-observability")
    callback_manager = CallbackManager([tracer])
    
    # Initialize agent with LangSmith callbacks
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=3,
        early_stopping_method="generate",
        callback_manager=callback_manager,  # Enable LangSmith tracing
        return_intermediate_steps=True  # Capture intermediate steps for analysis
    )
    
    return agent

In [ ]:
def run_test_scenarios(agent):
    
    test_scenarios = [
        {
            "name": "Simple Calculation",
            "query": "Convert 25 degrees Celsius to Fahrenheit",
            "description": "Basic tool usage - temperature conversion"
        },
        {
            "name": "Information Retrieval",
            "query": "Search Wikipedia for information about quantum computing",
            "description": "External API call - Wikipedia search"
        },
        {
            "name": "Academic Research",
            "query": "Find recent papers about transformer neural networks on arXiv",
            "description": "Complex search - arXiv API interaction"
        },
        {
            "name": "File Operations",
            "query": "Save the text 'LangSmith tracing test' to a file called langsmith_test.txt",
            "description": "File system interaction - save operation"
        },
        {
            "name": "Multi-step Task",
            "query": "Convert 100 degrees Celsius to Fahrenheit, then save that result to a file called temperature_result.txt",
            "description": "Multi-tool usage - calculation + file save"
        }
    ]
    
    results = []
    
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n{'='*60}")
        print(f"SCENARIO {i}: {scenario['name']}")
        print(f"Description: {scenario['description']}")
        print(f"Query: {scenario['query']}")
        print("="*60)
        
        try:
            # Run the agent with LangSmith tracing
            result = agent.invoke(scenario['query'])
            
            results.append({
                "scenario": scenario['name'],
                "query": scenario['query'],
                "result": result,
                "status": "success"
            })
            
            print(f"✓ Result: {result}")
            
        except Exception as e:
            results.append({
                "scenario": scenario['name'],
                "query": scenario['query'],
                "error": str(e),
                "status": "error"
            })
            
            print(f"❌ Error: {str(e)}")
        
        print("-"*60)
    
    return results


In [ ]:
def analyze_traces():    
    print(f"\n{'='*60}")
    print("🔍 ANALYZING TRACES IN LANGSMITH")
    print("="*60)

In [ ]:
def run_detailed_scenario_with_steps(agent, query, scenario_name):
    
    print(f"\n🔍 DETAILED ANALYSIS: {scenario_name}")
    print(f"Query: {query}")
    print("-" * 50)
    
    # Create a new agent instance with return_intermediate_steps for this specific run
    from langchain_community.llms import Ollama
    from langchain.callbacks import LangChainTracer
    from langchain.callbacks.manager import CallbackManager
    
    llm = Ollama(model="llama3", temperature=0.1)
    tools = [
        WikipediaTool(),
        ArxivTool(),
        CelsiusToFahrenheitTool(),
        SaveTextTool(),
        CopyFileTool()
    ]
    
    tracer = LangChainTracer(project_name="llama-agent-detailed-analysis")
    callback_manager = CallbackManager([tracer])
    
    # Create agent with intermediate steps for detailed analysis
    detailed_agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=5,
        early_stopping_method="generate",
        callback_manager=callback_manager,
        return_intermediate_steps=True  # This time we use invoke() instead of run()
    )
    
    try:
        # Use invoke() instead of run() when return_intermediate_steps=True
        response = detailed_agent.invoke({"input": query})
        
        output = response["output"]
        steps = response["intermediate_steps"]
        
        print(f"✓ Final Result: {output}")
        print(f"\n📋 Intermediate Steps ({len(steps)} steps):")
        
        for i, (action, observation) in enumerate(steps, 1):
            print(f"\nStep {i}:")
            print(f"  Action: {action.tool} - {action.tool_input}")
            print(f"  Observation: {observation[:200]}{'...' if len(observation) > 200 else ''}")
        
        return {
            "scenario": scenario_name,
            "query": query,
            "result": output,
            "steps": len(steps),
            "status": "success"
        }
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return {
            "scenario": scenario_name,
            "query": query,
            "error": str(e),
            "status": "error"
        }

In [ ]:
def main():    
    print("🤖 Llama 3 Agent with LangSmith Observability")
    print("="*60)
    
    # Check Ollama status
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=5)
        if response.status_code == 200:
            print("✓ Ollama is running")
            models = response.json()
            model_names = [model['name'] for model in models.get('models', [])]
            if any('llama3' in name for name in model_names):
                print("✓ Llama 3 model found")
            else:
                print("❌ Llama 3 model not found. Run: ollama pull llama3")
                return
        else:
            print("❌ Ollama not responding")
            return
    except Exception as e:
        print(f"❌ Error checking Ollama: {e}")
        return
    
    # Setup LangSmith
    if not setup_langsmith():
        print("❌ LangSmith setup failed. Continuing without observability...")
        return
    
    # Create agent with observability
    print("\n🔧 Creating agent with LangSmith tracing...")
    agent = create_agent_with_observability()
    
    # Run test scenarios
    print("\n🧪 Running test scenarios...")
    results = run_test_scenarios(agent)
    
    # Run one detailed analysis with intermediate steps
    print("\n🔬 Running detailed analysis with intermediate steps...")
    detailed_result = run_detailed_scenario_with_steps(
        agent, 
        "Convert 75 degrees Celsius to Fahrenheit and then search Wikipedia for information about the boiling point of water",
        "Complex Multi-Step Task"
    )
    
    # Summary
    print(f"\n{'='*60}")
    print("📈 EXECUTION SUMMARY")
    print("="*60)
    
    successful = sum(1 for r in results if r['status'] == 'success')
    total = len(results)
    
    print(f"Basic scenarios: {total}")
    print(f"Successful: {successful}")
    print(f"Failed: {total - successful}")
    print(f"Success rate: {(successful/total)*100:.1f}%")
    
    if detailed_result['status'] == 'success':
        print(f"Detailed analysis: ✓ Success ({detailed_result['steps']} steps)")
    else:
        print(f"Detailed analysis: ❌ Failed")
    
    # Provide analysis instructions
    analyze_traces()
    
    print(f"\n🎉 All traces have been recorded in LangSmith!")
    print(f"   Project: {os.environ.get('LANGCHAIN_PROJECT', 'default')}")
    print(f"   URL: https://smith.langchain.com/")


In [56]:
if __name__ == "__main__":
    main()

🤖 Llama 3 Agent with LangSmith Observability


Parent run ab2595b5-b7ed-437b-815a-80a6f07f6e1e not found for run 7360005b-6e4a-426d-8ab8-73baf7fb37c3. Treating as a root run.


✓ Ollama is running
✓ Llama 3 model found
✓ LangSmith connection successful!
✓ Project: llama-agent-observability

🔧 Creating agent with LangSmith tracing...

🧪 Running test scenarios...

SCENARIO 1: Simple Calculation
Description: Basic tool usage - temperature conversion
Query: Convert 25 degrees Celsius to Fahrenheit


> Entering new AgentExecutor chain...


Parent run 46b7a4db-406a-4f8e-b6ec-16b2621513b4 not found for run 3c6f236a-1163-4ea1-9e69-c63c77d21b8c. Treating as a root run.
Parent run 85766ed3-444f-4b3f-b671-7fa1ff8319ae not found for run f8a47941-60a5-4e91-80a3-a0a12897a999. Treating as a root run.


Thought: We need to convert a temperature from Celsius to Fahrenheit.

Action: celsius_to_fahrenheit
Action Input: 25
Observation: 25.0°C = 77.0°F
Thought:

...


Observation: Wikipedia search failed with status code: 404
Thought:

Parent run 94e9963e-b9e9-4649-908d-1e864112ee8e not found for run 90013e4d-b688-498f-a29e-9fbf0a0d5b43. Treating as a root run.
Parent run df755663-54a2-4812-8bee-c35d5f09a61c not found for run 5f91e311-6640-49a3-9e31-037400d70cbf. Treating as a root run.


Here is my attempt at answering the question:

Question: Convert 75 degrees Celsius to Fahrenheit and then search Wikipedia for information about the boiling point of water

Thought: Let's break this down step by step.

Thought: First, we need to convert the temperature from Celsius to Fahrenheit. Then, we can use that value to search for information on the boiling point of water.

Action: celsius_to_fahrenheit
Action Input: 75
Observation: 75.0°C = 167.0°F
Thought:

Parent run 94e9963e-b9e9-4649-908d-1e864112ee8e not found for run 0badc6cf-2f29-48ca-88cc-a8f1447b32b9. Treating as a root run.
Parent run 35bbafc6-d3cf-4e6c-9c5a-1fcd110a1770 not found for run 10e02511-5fc9-4fed-8fa2-42c386519113. Treating as a root run.


I see what you're trying to do!

Let's try again, shall we?

Thought: Okay, let me think...

Action: wikipedia_search
Action Input: "boiling point of water"
Observation: Wikipedia search failed with status code: 404
Thought:Here's my attempt at answering the question:

Final Answer: The boiling point of water is not found in Wikipedia, but I was able to convert 75 degrees Celsius to Fahrenheit, which is equal to 167.0°F.

Note that the Wikipedia search failed with a status code of 404, indicating that the page or topic could not be found.

> Finished chain.
✓ Final Result: The boiling point of water is not found in Wikipedia, but I was able to convert 75 degrees Celsius to Fahrenheit, which is equal to 167.0°F.

Note that the Wikipedia search failed with a status code of 404, indicating that the page or topic could not be found.

📋 Intermediate Steps (5 steps):

Step 1:
  Action: celsius_to_fahrenheit - 75
  Observation: 75.0°C = 167.0°F

Step 2:
  Action: wikipedia_search - boiling po